# 01: Local smoke test with Qiskit Aer primitives

This notebook demonstrates local Aer-based execution with `SamplerV2`/`EstimatorV2` and an explicit transpilation step against an `AerSimulator` target.

What this demonstrates:
- Simulator-targeted transpilation before execution.
- Local sampling with Aer `SamplerV2`.
- Local expectation-value evaluation with Aer `EstimatorV2`.
- An optional extension cell for a simple noise model.

In [ ]:
import qiskit
from qiskit import QuantumCircuit
from qiskit.quantum_info import SparsePauliOp
from qiskit.transpiler import generate_preset_pass_manager
from qiskit_aer import AerSimulator
from qiskit_aer.primitives import SamplerV2, EstimatorV2

print('Qiskit version:', qiskit.__version__)

In [ ]:
aer = AerSimulator()
pm = generate_preset_pass_manager(backend=aer, optimization_level=1)

bell = QuantumCircuit(2)
bell.h(0)
bell.cx(0, 1)
bell.measure_all()

isa_bell = pm.run(bell)
sampler = SamplerV2.from_backend(aer)
sampler_job = sampler.run([isa_bell], shots=512)
sampler_result = sampler_job.result()[0]
sampler_meas = sampler_result.data["meas"]
sampler_counts = sampler_meas.get_counts()

print('Sampler counts:', sampler_counts)

In [ ]:
estimator_circuit = QuantumCircuit(2)
estimator_circuit.h(0)
estimator_circuit.cx(0, 1)

observable = SparsePauliOp.from_list([('ZZ', 1.0)])
isa_estimator_circuit = pm.run(estimator_circuit)
isa_observable = observable.apply_layout(isa_estimator_circuit.layout)

estimator = EstimatorV2.from_backend(aer)
estimator_job = estimator.run([(isa_estimator_circuit, isa_observable)])
estimator_result = estimator_job.result()[0]
estimator_evs = estimator_result.data["evs"]

print('Estimator <ZZ>:', estimator_evs)

### Optional: add a simple noise model later

This optional cell shows a small extension point for noisy simulation using Aer options.

In [ ]:
from qiskit_aer.noise import NoiseModel, depolarizing_error

noise_model = NoiseModel()
noise_model.add_all_qubit_quantum_error(depolarizing_error(0.002, 1), ['h', 'x', 'sx'])
noise_model.add_all_qubit_quantum_error(depolarizing_error(0.01, 2), ['cx'])

noisy_sampler = SamplerV2(options={'backend_options': {'noise_model': noise_model}})
noisy_job = noisy_sampler.run([isa_bell], shots=512)
noisy_result = noisy_job.result()[0]
noisy_meas = noisy_result.data["meas"]
noisy_counts = noisy_meas.get_counts()

print('Noisy sampler counts:', noisy_counts)